<h1><center>Laboratorio 5: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Diego Bartolucci
- Nombre de alumno 2: Pilar Nilo


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/DiegoBarto01/MDS7202-pili-barto.git)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Jueves a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Lab_5/'
except:
    print('Ignorando conexión drive-colab')

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [ ]:
# Inserte su código aquí
import pickle
path_retail= path + "/online_retail_data.pickle"
with open(path_retail, 'rb') as file:
    object_pkl = pickle.load(file)
df_retail = object_pkl
df_retail

### 1. Función para explorar características [5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [3 puntos]
- Imprima un conteo de datos nulos por variable [2 puntos]

NOTA: Para generar los gráficos es **OBLIGATORIO** el uso de plotly.

**Respuesta:**



**Importante!!!!**
> La función explore data en un principio se realizó solo para Price y Quantity, por lo que para la sección 4.2 al hacer explore_data con otras variables como Length, Recency etc, no funcionaba. Por lo anterior se decide generalizar la función explore_data, a la cual se le entrega el df y las variables que deseo plotear o visualizar en un histograma.







In [4]:
def explore_data(dataframe_in, column_name):
    #recorro las variables que deseo explorar
    for x in column_name:
      if x in dataframe_in.columns: #se verifica que estén en el df
          #se crea el histograma usando Plotly de cada variable
          hist = px.histogram(dataframe_in, x=x, title=f"Histograma de la variable {x}")
          hist.show()
      else:
          print(f"La columna '{x}' no existe en el DataFrame.") #caso en q no exista

    #2.imprima un conteo de datos nulos por variable
    count_null=dataframe_in.isnull().sum()
    print(f'Cantidad de datos null en cada columna:')
    display(count_null)

In [ ]:
explore_data(df_retail,['Price','Quantity'])

### 2. Eliminando outliers [10 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

Hint: tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).
>**Nota: No modificar el método set_output de la clase IQR**

**Respuesta:**

In [6]:
#from _typeshed import NoneType
class IQR(BaseEstimator, TransformerMixin):

  def __init__(self, lam):
    #Completar código aquí
    self.lim_inf=None
    self.lim_sup=None
    self.lam=lam

  def fit(self, X, y=None):
    #Completar código aquí
    q1=X.quantile(0.25)
    q3=X.quantile(0.75)
    ric=q3-q1
    self.lim_inf=q1-self.lam*ric
    self.lim_sup=q3+self.lam*ric
    return self


  def transform(self, X):
    #Completar código aquí
    for i in X.columns:
      X[i]= X[i].clip(lower=self.lim_inf[i],upper=self.lim_sup[i])
    return X

  def set_output(self,transform='default'):
    #No modificar esta función
    return self

#### 2.2 Creación del Pipeline [5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [1 punto]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [1 punto]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [1 punto]


HINT: El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.
>**Nota: Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.**

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

buscar un lambda que funcione bn

In [ ]:
df_retail.columns

In [ ]:
# Definicion las variables que pasarán por cada pipeline
numerical_columns = ['Price','Quantity']
categorical_columns = ['Invoice','StockCode','Description','InvoiceDate','Customer ID','Country']

# Definicion del pipeline
numeric_transformations = Pipeline([('iqr_numerical', IQR(lam=5))])

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations,numerical_columns),
                                        ('categorical', 'passthrough',categorical_columns)
                                        ],
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos
df_iqr = column_transformer.fit_transform(df_retail)

# Gráficos
explore_data(df_retail,['Price','Quantity'])
explore_data(df_iqr,['Price','Quantity'])

*Reporte los cambios observados aquí.*

Los cambios observados son únicamente en los histogramas ya que la cantidad de datos null en cada columna se mantienen igual.

Por lo anterior, mejoró la visualización de los valores en los histogramas, es decir que no todos se concentran en un solo intervalo sino que es posible detectar como fluctua dependiendo de la cantidad y el precio con mayor especificidad ya que los intervalos del eje X son menos espaciados el uno del otro, a diferencia de los dos primeros histogramas que poseen intervalos más espaciados y solo se ve una barra, solo es posible ver a detalle cuando se realiza un zoom.

### 3. Agregando un imputer al pipeline [10 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [1 punto]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [1 punto]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [2 puntos]

>**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [9]:
# Pregunta 1
from sklearn.impute import SimpleImputer
#Se agrega el mode imputer por categoría más frecuente
categoric_transformations = Pipeline([('mode_imputer', SimpleImputer(strategy='most_frequent'))])

In [10]:
# Pregunta 2
#Se agrega el paso para que los datos se imputen según el promedio
numeric_transformations.steps.append(('mean_imputer', SimpleImputer(strategy='mean')))

In [11]:
# Pregunta 3
#Con los cambios anteriores se aplica un ColumnTrasnformer
column_transformer = ColumnTransformer(
    [('numeric', numeric_transformations, numerical_columns),
     ('categorical', categoric_transformations, categorical_columns)],
    verbose_feature_names_out=False
)

column_transformer.set_output(transform='pandas')
df_mean_imputer = column_transformer.fit_transform(df_retail)

In [ ]:
# Pregunta 4
#Se grafican
explore_data(df_iqr,['Price','Quantity'])
explore_data(df_mean_imputer,['Price','Quantity'])

Las principal diferencia en ambas variables se tiene que ya no hay valores nulos en ninguna columna por lo que los valores de las columnas son menores.

In [14]:
# Pregunta 5
from sklearn.impute import KNNImputer
#Se repiten los pasos anteriores pero imputando los datos con KNN
numeric_transformations.steps[-1] = ('knn_imputer', KNNImputer(n_neighbors=3))

column_transformer.set_output(transform='pandas')
df_knn_imputer = column_transformer.fit_transform(df_retail)

In [ ]:
# Pregunta 6
#Se grafica
explore_data(df_iqr,['Price','Quantity'])
explore_data(df_knn_imputer,['Price','Quantity'])

Se observa que en el caso del knn imputer, existen más barras, tanto en la variable Price, como en la variable Quantity. Esto se debe a que la imputación se basa en los vecinos más cercanos (y no solamente con la media, como se hace en el caso anterior).

In [ ]:
# Pregunta 7
#Se comparan los gráficos finales
explore_data(df_mean_imputer,['Price','Quantity'])
explore_data(df_knn_imputer,['Price','Quantity'])

El método de imputación que es mejor corresponde al KNN, ya que el algoritmo considera las demás variables para la estimación, por lo que corresponderían a valores más exactos.

### 4. Creación de nuevas features [20 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [10 puntos]
(2 puntos por cada custom feature)

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [ ]:
query_max=df_retail[['Customer ID','InvoiceDate']].groupby('Customer ID').max()
ultima_visita=df_retail['InvoiceDate'].max()
R=(ultima_visita-query_max['InvoiceDate']).dt.days
hoy= pd.Timestamp.now()
R1=(hoy-query_max['InvoiceDate']).dt.days

In [ ]:
R1

In [ ]:
def custom_features(dataframe_in):
  dataframe_in.sort_values(['Customer ID','InvoiceDate'],inplace=True)
  #Length
  query_min=dataframe_in[['Customer ID','InvoiceDate']].groupby('Customer ID').min()
  query_max=dataframe_in[['Customer ID','InvoiceDate']].groupby('Customer ID').max()
  query_length=query_max['InvoiceDate']-query_min['InvoiceDate']
  L=query_length.dt.days
  #Recency
  hoy= pd.Timestamp.now()
  R=(hoy-query_max['InvoiceDate']).dt.days
  #Monetary
  query_mean=dataframe_in[['Customer ID','Price']].groupby('Customer ID').mean()
  M=query_mean['Price']
  #Frecuency
  count_visit=dataframe_in[['Customer ID','InvoiceDate']].groupby('Customer ID').count()
  F=count_visit['InvoiceDate']
  #Periodicity
  #---IVT
  df_ivt=dataframe_in.copy()
  df_ivt=df_ivt.sort_values(['Customer ID','InvoiceDate']).drop_duplicates(subset=['Customer ID','InvoiceDate'])
  #---agregamos cosas
  df_ivt['IVT']=df_ivt.groupby('Customer ID')['InvoiceDate'].diff()
  df_ivt['Invoice'] = df_ivt['Invoice'].shift(-1)
  query= df_ivt[['Customer ID','IVT']].groupby('Customer ID').std().sort_index()
  p=query['IVT'].dt.days
  p=p.fillna(0)

  #armamos el df de las metricas

  df_metrics=pd.DataFrame({
                           'Length':L.values,
                           'Recency':R.values,
                           'Frequency':F.values,
                           'Monetary':M.values,
                           'Periodicity':p.values}, index=L.index)
  return df_metrics





In [ ]:
custom_features(df_retail)

In [ ]:
#pasar a dataframe
def paso_a_df(array):
    array = pd.DataFrame(array, columns=['Price', 'Quantity', 'Invoice',
                                 'StockCode', 'Description', 'InvoiceDate',
                                 'Customer ID', 'Country'])
    return array

#### 4.2 Agregando las custom features [10 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [1 punto]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [1 punto]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [ ]:
#1 pipeline que encapsule columntransformer y calcule LRMFP
retail_pipeline = Pipeline([('col_transformer', column_transformer),
                            ('to_dataframe', FunctionTransformer(paso_a_df, validate=False)),
                            ('custom_features', FunctionTransformer(custom_features, validate=False))])

In [ ]:
#2 aplicar pipeline actualizado a los datos, creando un nuevo dataframe df_custom
df_custom = retail_pipeline.fit_transform(df_retail)

In [ ]:
explore_data(df_custom,['Length','Recency','Monetary','Frequency','Periodicity'])

Insight

>según el grafico hay bastantes clientes fieles, por lo que es ideal mantener esa clientela incentivandola con descuentos, sorteos, etc.
En cambio, Recency tiene baja distribución, por lo que para incentivar que los clientes aumenten sus visitas se deben realizar actividades en el retail donde el cliente pueda ganar productos.
Monetary nos dice los intervalos minimos y maximos de lo que gastan en promedio, por lo que se sugiere realizar activaciones del estilo 'Ofertas relampagos', 'oferta de temporada', etc.





### 5. MinMax Scaler [10 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [17]:
class MinMax(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.min = None
        self.max = None

    def fit(self,X):
        self.min = X.min()
        self.max = X.max()
        return self

    def transform(self,X):
        return (X - self.min) / (self.max - self.min)

    def set_output(self,transform='default'):
        #No modificar este método
        return self



#### 5.2 Incorporando MinMax al pipeline [5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [1 punto]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [1 punto]
- Reportar los cambios observados en la distribución de las variables.  [2 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

In [18]:
# Paso 1
numeric_transformations.steps.append(('minmax', MinMax()))

In [ ]:
# Paso 2
column_transformer = ColumnTransformer([('numeric', numeric_transformations, numerical_columns),
                                        ('categorical', categoric_transformations, categorical_columns)
                                        ],
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')
df_minmax = column_transformer.fit_transform(df_retail)

df_minmax

In [ ]:
# Paso 3
explore_data(df_retail,['Price','Quantity'])
explore_data(df_minmax,['Price','Quantity'])

Se observa de mejor manera la distribución de las variables en df_minmax que en df_retail, ya que se normalizaron según el maximo y el mínimo de estas.

### 6. Pregunta teórica [5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

Los pipelines organizan el preprocesamiento de datos, asegurando que cada paso se aplique de manera consistente y reproducible en diferentes datasets. Esto mejora la eficiencia y evita errores manuales, permitiendo un flujo de trabajo más limpio y automatizado.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>